In [4]:
from logic.getting import *
from logic.scaler import *
from logic.supporting_functions import *
from logic.final_preproc_function import *
from logic.windows_preproc import *
from logic.stats import var
from logic.gbm import gbm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
import cvxpy as cp





In [3]:
stocks = ['AAPL', 'AMT', 'MSFT', 'CAT', 'GS', 'JNJ', 'MCD', 'NEE', 'PG', 'XOM']
x0 = [1/len(stocks) for i in range(len(stocks))]
stocks_df = [0 for i in range(len(stocks))]

for i in range(len(stocks)):
    stock = get_price_data_raw(stocks[i])
    rename_columns(stock)
    stock = stock.sort_values(by='datetime')
    stocks_df[i] = stock 

/root/.pyenv/versions/3.10.6/envs/portfolio_manager/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


KeyboardInterrupt: 

In [4]:
def portfolio_prices(stocks: list, weights: list):
    df = stocks[0].filter(['datetime', 'close'])
    df['close'] = df['close'] * weights[0]
    
    for stock, weight in zip(stocks[1:], weights[1:]):
        df = df.merge(stock.filter(['close', 'datetime']), on= 'datetime', how= 'inner')
        df['close_x'] = df['close_x'] + df['close_y']* weight
        df = df.rename(columns = {'close_x': 'close'})
        df = df.drop(columns = 'close_y')
    return df


def objective(X):
    return portfolio_prices(stocks_df, X)['close'][49000] - portfolio_prices(stocks_df, X)['close'][49100]

def constraint(X):
    simulations = gbm(portfolio_prices(stocks_df, X)['close'], 29000, n=1000, T=100, M=2000)
    return 5 - var(simulations, 95)

n = len(x0)
con = [{'type': 'ineq', 'fun': constraint}]
con.append({'type': 'eq', 'fun':  lambda x: sum(x[:n]) - 1})

bounds = Bounds([0]*len(x0), [1]*len(x0))

# for i in range(n):
#     con.append({'type': 'ineq', 'fun': lambda x, i=i: 1 - x[i]})
#     con.append({'type': 'ineq', 'fun': lambda x, i=i: x[i]})
    
    

# con = [{'type': 'ineq', 'fun': constraint},
#        {'type': 'eq', 'fun':  lambda x: x[0] + x[1] + x[2] - 1},
#        {'type': 'ineq', 'fun':  lambda x: 1 - x[0]},
#        {'type': 'ineq', 'fun':  lambda x: 1 - x[1]},
#        {'type': 'ineq', 'fun':  lambda x: 1 - x[2]}
#       ]

In [5]:
solution = minimize(objective, x0, constraints=con, method='SLSQP', bounds=bounds, options={'maxiter': 100, 'disp': True})
solution

Iteration limit reached    (Exit mode 9)
            Current function value: -5.594998700845636
            Iterations: 100
            Function evaluations: 1533
            Gradient evaluations: 100


 message: Iteration limit reached
 success: False
  status: 9
     fun: -5.594998700845636
       x: [ 1.858e-08  1.000e+00  4.845e-08  2.073e-09  6.289e-09
            2.066e-08  3.083e-09  9.997e-08  2.807e-08  2.100e-08]
     nit: 100
     jac: [-3.352e+00 -5.595e+00  1.675e+00  2.549e+00  7.860e-01
           -1.729e+00 -1.418e+00 -7.800e-01 -2.094e+00  2.778e+00]
    nfev: 1533
    njev: 100

In [ ]:
portfolio_prices(['AAPL', 'AMT'], [1, 2])

# cvxpy

In [279]:
n_assets = len(stocks_df)  # Number of assets

returns = np.zeros((100, n_assets))

for i, df in enumerate(stocks_df):
    returns[:, i] = df['close'].pct_change().dropna().values[4900:5000]

In [296]:
import cvxpy as cp
import numpy as np

# Number of assets (stocks)
n_assets = 10

# Example returns matrix (100 time periods x 10 assets)
returns = np.random.randn(100, n_assets)  # Replace with actual returns data

# Define the decision variable for portfolio weights
weights = cp.Variable(n_assets)

# Calculate the portfolio returns for the last 100 time periods
portfolio_returns = returns @ weights

# Objective: Maximize the sum of portfolio returns over the 100 time periods
objective = cp.Maximize(cp.sum(portfolio_returns))

# Define constraints
constraints = [
    cp.sum(weights) == 1,  # All weights must sum to 1
    weights >= 0,          # No short positions (non-negative weights)
    weights <= 1           # Weights cannot exceed 1
]

# Adding a VaR constraint
# Example VaR function that takes portfolio returns as input
def var_constraint(portfolio_returns):
    simulations = gbm(portfolio_return)
    return var(portfolio_returns, 95)

def constraint(X):
    simulations = gbm(portfolio_prices(stocks_df, X)['close'], 29000, n=1000, T=100, M=2000)
    return 5 - var(simulations, 95)

# Use cp.norm or another suitable function to approximate/express the VaR constraint in a convex form
# Here, we assume an approximate method:
# VaR_estimate = cp.norm(portfolio_returns, p=2)  # This is a placeholder; use your actual method

# # Ensure VaR is less than 5%
# constraints.append(VaR_estimate <= 0.05)

# # Define and solve the optimization problem
# problem = cp.Problem(objective, constraints)
# result = problem.solve()

# # Check the solver status
# print(f"Solver status: {problem.status}")

# # Get the optimized weights if the problem was solved optimally
# if problem.status == cp.OPTIMAL:
#     optimal_weights = weights.value
#     print("Optimized Weights:", optimal_weights)
# else:
#     print("Optimization did not converge to an optimal solution.")

var_constraint(portfolio_returns)

AttributeError: 'MulExpression' object has no attribute 'sort'

# Other

In [ ]:
aapl = get_price_data_raw('AAPL')
amt = get_price_data_raw('AMT')
cat = get_price_data_raw('CAT')
gs = get_price_data_raw('GS')
jnj = get_price_data_raw('JNJ')
mcd = get_price_data_raw('MCD')
msft = get_price_data_raw('MSFT')
nee = get_price_data_raw('NEE')
pg = get_price_data_raw('PG')
xom = get_price_data_raw('XOM')

In [ ]:
aapl = aapl.rename(columns = {'datetime' : 'timestamp_field_0'})
pg = pg.rename(columns = {'datetime' : 'timestamp_field_0'})

In [ ]:
aapl = aapl.sort_values(by='timestamp_field_0')
amt = amt.sort_values(by='timestamp_field_0')
cat = cat.sort_values(by='timestamp_field_0')
gs = gs.sort_values(by='timestamp_field_0')
jnj = jnj.sort_values(by='timestamp_field_0')
mcd = mcd.sort_values(by='timestamp_field_0')
msft = msft.sort_values(by='timestamp_field_0')
nee = nee.sort_values(by='timestamp_field_0')
pg = pg.sort_values(by='timestamp_field_0')
xom = xom.sort_values(by='timestamp_field_0')

In [ ]:
rename_columns()

In [ ]:
datetime_column = aapl.select_dtypes(include=['datetime']).columns
datetime_column

In [ ]:
def portfolio_prices(stocks: list, weights: list):
    df = stocks[0].filter(['timestamp_field_0', 'close'])
    df['close'] = df['close'] * weights[0]
    for stock, weight in zip(stocks[1:], weights[1:]):
        df = df.merge(stock.filter(['close', 'timestamp_field_0']), on= 'timestamp_field_0', how= 'inner')
        df['close_x'] = df['close_x'] + df['close_y']* weight
        df = df.rename(columns = {'close_x': 'close'})
        df = df.drop(columns = 'close_y')
    return df

In [ ]:
df = aapl.filter(['timestamp_field_0', 'close'])
df = df.merge(amt.filter(['close', 'timestamp_field_0']), on= 'timestamp_field_0', how= 'inner')
df['close_x'] = df['close_x'] + df['close_y']
df = df.rename(columns = {'close_x': 'close'})
df = df.drop(columns = 'close_y')
df = df.merge(cat.filter(['close', 'timestamp_field_0']), on= 'timestamp_field_0', how= 'inner')
df

In [ ]:
portfolio_prices([aapl, amt, cat, gs, jnj, mcd, msft, nee, pg, xom], [20.1, -10.1, 0.2, 0.2, 0.3, 0.3, 0.4, 0.4, 0.5, 0.5])